# Measurement optimization in single qubit system (single parameter)

This is the source codes for the example discussed in Sec. VI in Ref. [1].

A single qubit system whose dynamics is governed by
\begin{align}
\partial_t\rho=-i[H, \rho]+ \gamma_{+}\left(\sigma_{+}\rho\sigma_{-}-\frac{1}{2}\{\sigma_{-}\sigma_{+},\rho\}\right)+ \gamma_{-}\left(\sigma_{-}\rho\sigma_{+}-\frac{1}{2}\{\sigma_{+}\sigma_{-},\rho\}\right),
\end{align}

where $H = \frac{1}{2}\omega \sigma_3$ is the free Hamiltonian with $\omega$ the frequency, $\sigma_{\pm}=(\sigma_1 \pm \sigma_2)/2$ and $\gamma_{+}$, $\gamma_{-}$ are decay rates.
Here $\sigma_{i}$ for $(i=1,2,3)$ is the  Pauli matrix.

In this case, the probe state is taken as $\frac{1}{\sqrt{2}}(|0\rangle +|1\rangle)$, $|0\rangle$ $(|1\rangle)$ is the 
eigenstate of $\sigma_3$ with respect to the eigenvalue $1$ $(-1)$. Here we use algorithms to obtain the optimal 
projective measurements.

[1] M. Zhang et al., QuanEstimation: an open-source toolkit for quantum parameter estimation,
arXiv:2205.15588.

In [1]:
from quanestimation import *
import numpy as np

# initial state
rho0 = 0.5*np.array([[1., 1.], [1., 1.]])
# free Hamiltonian
omega = 1.0
sx = np.array([[0., 1.], [1., 0.]])
sy = np.array([[0., -1.j], [1.j, 0.]]) 
sz = np.array([[1., 0.], [0., -1.]])
H0 = 0.5*omega*sz
# derivative of the free Hamiltonian on omega
dH = [0.5*sz]
# dissipation
sp = np.array([[0., 1.], [0., 0.]])  
sm = np.array([[0., 0.], [1., 0.]]) 
decay = [[sp, 0.], [sm, 0.1]]
# generation of a set of POVM basis
dim = np.shape(rho0)[0]
POVM_basis = SIC(dim)
# time length for the evolution
tspan = np.linspace(0., 10., 25)

## Projective measurement optimization

In [13]:
# # generation of the entry of `measurement0`
# C = [[] for i in range(dim)] 
# for i in range(dim):
#     r_ini = 2*np.random.random(dim)-np.ones(dim)
#     r = r_ini/np.linalg.norm(r_ini)
#     phi = 2*np.pi*np.random.random(dim)
#     C[i] = [r[j]*np.exp(1.0j*phi[j]) for j in range(dim)]
# C = np.array(gramschmidt(C))
# measurement0 = [C for _ in range(10)]


# measurement optimization algorithm: DE
DE_paras = {"p_num":10, "measurement0":[], "max_episode":1000, \
            "c":1.0, "cr":0.5, "seed":1234}
m = MeasurementOpt(mtype="projection", minput=[], savefile=False, \
                   method="DE", **DE_paras)

# # measurement optimization algorithm: PSO
# PSO_paras = {"p_num":10, "measurement0":[], "max_episode":[1000,100], \
#              "c0":1.0, "c1":2.0, "c2":2.0, "seed":1234}
# m = MeasurementOpt(mtype="projection", minput=[], savefile=False, \
#                    method="PSO", **PSO_paras)

In [14]:
# input the dynamics data
m.dynamics(tspan, rho0, H0, dH, decay=decay, dyn_method="expm")
# objective function: CFI
m.CFIM()

In [15]:
# load the measurements
M = np.load("measurements.npy")[-1]

## Find the optimal linear combination of an input measurement

In [3]:
# the number of operators of the output measurement
M_num = 2

# # generation of the entry of `measurement0`
# B = np.array([np.random.random(len(POVM_basis)) for i in range(M_num)])
# measurement0 = [B for _ in range(10)]

# measurement optimization algorithm: DE
DE_paras = {"p_num":10, "measurement0":[], "max_episode":1000, \
            "c":1.0, "cr":0.5, "seed":1234}
m = MeasurementOpt(mtype="input", minput=["LC", POVM_basis, M_num], \
                   savefile=False, method="DE", **DE_paras)

# # measurement optimization algorithm: PSO
# PSO_paras = {"p_num":10, "measurement0":[], \
#              "max_episode":[1000,100], "c0":1.0, \
#              "c1":2.0, "c2":2.0, "seed":1234}
# m = MeasurementOpt(mtype="input", minput=["LC", POVM_basis, M_num], \
#                    savefile=False, method="PSO", **PSO_paras)

# # measurement optimization algorithm: AD
# AD_paras = {"Adam":False, "measurement0":[], "max_episode":300, \
#             "epsilon":0.01, "beta1":0.90, "beta2":0.99}
# m = MeasurementOpt(mtype="input", minput=["LC", POVM_basis, M_num], \
#                    savefile=False, method="AD", **AD_paras)

In [4]:
# input the dynamics data
m.dynamics(tspan, rho0, H0, dH, decay=decay, dyn_method="expm")
# objective function: CFI
m.CFIM()

In [6]:
# load the measurements
M = np.load("measurements.npy")[-1]

## Find the optimal rotated measurement of an input measurement

In [5]:
M_num = dim

# generation of the entry of `measurement0`
s = np.random.random(dim**2)
measurement0 = [s for _ in range(10)]

In [16]:

# measurement optimization algorithm: DE
DE_paras = {"p_num":10, "measurement0":[], "max_episode":1000, \
            "c":1.0, "cr":0.5, "seed":1234}
m = MeasurementOpt(mtype="input", minput=["rotation", POVM_basis], \
                   savefile=False, method="DE", **DE_paras)

# # measurement optimization algorithm: PSO
# PSO_paras = {"p_num":10, "measurement0":[], \
#              "max_episode":[1000,100], "c0":1.0, \
#              "c1":2.0, "c2":2.0, "seed":1234}
# m = MeasurementOpt(mtype="input", minput=["rotation", POVM_basis], \
#                    savefile=False, method="PSO", **PSO_paras)

# # measurement optimization algorithm: AD
# AD_paras = {"Adam":False, "measurement0":[], "max_episode":300, \
#             "epsilon":0.01, "beta1":0.90, "beta2":0.99}
# m = MeasurementOpt(mtype="input", minput=["rotation", POVM_basis], \
#                    savefile=False, method="AD", **AD_paras)

In [17]:
# input the dynamics data
m.dynamics(tspan, rho0, H0, dH, decay=decay, dyn_method="expm")
# objective function: CFI
m.CFIM()

In [21]:
# load the measurements
M = np.load("measurements.npy")[-1]